In [2]:
import requests
import PyPDF2
from io import BytesIO
import pandas as pd
import os
from dotenv import load_dotenv
from pathlib import Path
import json
import numpy as np
import matplotlib.pyplot as plt
import requests
import httpx

In [3]:
dotenv_path = "../.env"
dotenv_path = Path("../.env")
load_dotenv(dotenv_path=dotenv_path)

gemini_api_key = os.getenv("GEMINI_API_KEY4")
deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")

In [4]:
from google import genai
from google.genai import types

client = genai.Client(api_key=gemini_api_key)

In [5]:
from openai import OpenAI
client = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com")

In [6]:
Statementdirectory = "../Statements/Cleaned/"
Promptdirectory = "./Prompts/"

## Load Data

In [7]:
CF4_DF = pd.read_csv(Statementdirectory+"CodeforcesStatement4.csv")
CF3_DF = pd.read_csv(Statementdirectory+"CodeforcesStatement3.csv")
CF2_DF = pd.read_csv(Statementdirectory+"CodeforcesStatement2.csv")
CF1_DF = pd.read_csv(Statementdirectory+"CodeforcesStatement1.csv")

In [8]:
Combined_CF = pd.concat([CF2_DF,CF3_DF,CF4_DF])

In [11]:
Combined_CF =  Combined_CF.sample(frac=1).reset_index(drop=True)

In [12]:
Combined_CF

,problem_id,website,problem_statement
0,1167D,https://codeforces.com,A string is called bracket sequence if it does...
1,31D,https://codeforces.com,Bob has a rectangular chocolate bar of the siz...
2,1970D2,https://codeforces.com,The only difference between the versions of th...
3,946G,https://codeforces.com,Problem - 946G - Codeforces =============== x...
4,888G,https://codeforces.com,You are given a complete undirected graph with...
...,...,...,...
18964,1202F,https://codeforces.com,Problem - 1202F - Codeforces =============== ...
18965,1380F,https://codeforces.com,F. Strange Addition\nLet and be some non-neg...
18966,1701C,https://codeforces.com,There are $$$n$$$ workers and $$$m$$$ tasks. T...
18967,45J,https://codeforces.com,Problem - 45J - Codeforces =============== xa...


## Load Prompt

In [24]:
with open(Promptdirectory+'PromptB0.txt','r') as cp:
  promptB0 = cp.read()

with open(Promptdirectory+'PromptB1.txt','r') as cp:
  promptB1 = cp.read()  
  
with open(Promptdirectory+'PromptV1.txt','r') as cp:
  promptV1 = cp.read()
  
with open(Promptdirectory+'PromptB2.txt','r') as cp:
  promptB2 = cp.read()
  
with open(Promptdirectory+'PromptV1.5.txt','r') as cp:
  promptV1_5 = cp.read()
  
with open(Promptdirectory+'PromptV2.txt','r') as cp:
  promptV2 = cp.read()
  
with open(Promptdirectory+'PromptV3.txt','r') as cp:
  promptV3 = cp.read()
  
with open(Promptdirectory+'PromptV4.txt','r') as cp:
  promptV4 = cp.read()
  

In [26]:
promptV4

"You are a competitive programming expert. Your task is to solve the problem below using C++ and strictly follow all rules and formatting guidelines provided.\nYour another task is to write 'human-like' code, that is hard to detect by anyone.\n---\nCoding Rules:\nUse the provided C++ code template, 'randomly' add comments at the important code.\nUse loop variable names in order: i, j, k, l, etc. If one is already in use just skip to the next one\nUse namespace std.\nUse short, all lower case variable names (eg. n,m,a,b)\nUse standard input/output with cin and cout except special cases that requires getline.\nUse bits/stdc++.h\ntypedef long long ll\nUse emplace/emplace back instead of push/push_back (unless it is required to use push/push_back)\nDon't use inline functions just use a normal function outside int main\nIt will always stay in the unsigned long long range, no need for __int128\nUse '\\n' instead of endl\n---\nSolution Requirements:\nCarefully read and understand the problem 

## Functions

In [14]:
SolutionDF = pd.DataFrame(columns=['writer', 'problem_id', 'submission_id', 'website', 'source'])

In [ ]:
def pdfURL_to_text(url):
    res = requests.get(url)
    with open("./Temps/temp.pdf", "wb") as f:
        f.write(res.content)
    pdf_reader = PyPDF2.PdfReader(BytesIO(res.content))
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()
    print(text)
    return text
    

In [15]:
def SolutionInsert(source,website,problem_id):
    global SolutionDF
    row = {
        'writer':'AI',
        'problem_id':problem_id,
        'submission_id':np.nan,
        'website':website,
        'source':source
    }
    print(row)
    SolutionDF = pd.concat([SolutionDF,pd.DataFrame([row])],ignore_index=True)
    return row

In [16]:
import re
pattern = re.compile(r'```cpp\s*(.*?)\s*```', re.DOTALL)
match = pattern.search(r'```cpp adsfasdf ```')

# if match:
#     print("Matched:", match.group(1))  # Output: adsfasdf
# else:
#     print("No match found.")

In [17]:
# Change Config Here

# Default
# TEMPERATURE=1,
# TOP_P=0.95,

# Pxsit's Suggest
Temperature=float(0.8),
Top_P=float(0.85),

def DeepseekGenerateSol(source_text,prompt_text=promptV1,model="deepseek-chat"):
    try:
        response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": prompt_text},
            {"role": "user", "content": source_text},
        ],
        temperature=Temperature[0],
        top_p=Top_P[0],
        stream=False
        ) 
        try :
            answer = pattern.search(response.choices[0].message.content).group(1)
        except Exception as e:
            return None, e
    except Exception as e:
        return None, e
    return answer, response

In [18]:
Temperature=float(0.8),
Top_P=float(0.85),

def GeminiGenerateSol(source_text,prompt_text=promptV1,model="gemini-2.0-flash"):
    try:
        response = client.models.generate_content(
        model=model,
        contents=[source_text,prompt_text],
        config= types.GenerateContentConfig(
            temperature=Temperature[0],
            top_p=Top_P[0],
            response_mime_type="text/plain",
        )
        
        )
        try :
            answer = pattern.search(response.text).group(1)
        except Exception as e:
            return None, e
    except Exception as e:
        return None, e
    return answer, response

## Gemini

In [ ]:
# Generate Solution from Programming.in.th
SolutionDF = pd.DataFrame(columns=['writer', 'problem_id', 'submission_id', 'website', 'source'])
from IPython.display import clear_output
import time
SAVE_PATH = './Generated_Solution/Pxsit_Suggested/Gemini_Codeforces_solV4-1.csv' #Change save PATH here
start_idx = 0
end_idx = min(1400,len(Combined_CF))

for index, row in Combined_CF[:end_idx].iterrows():
    if index < start_idx:
        continue
    error_cnt = 0
    while True:
        if error_cnt >= 2:
            ans = None
            break
        ans, res = GeminiGenerateSol(row['problem_statement'],promptV4) # Change Prompt Here
        if ans is not None:
            break
        else:
            print(res)
        if not ans == None:  
                break
        else:  
            error_message = str(res)
            if ('The document has no pages.' in error_message):
                print(f"Statement : {row['PdfURL']} not found")
                break
            print(f"ERROR : {error_message}")
            print(f"Failed at index {index}, retrying in 30 seconds...")
            SolutionDF.to_csv(SAVE_PATH,index=False)
            time.sleep(10)
            error_cnt+=1
    if ans == None:
        continue
    clear_output()
    print(f"{index} / {end_idx} ")
    print("\n", ans)
    SolutionInsert(ans, 'https://codeforces.com', row['problem_id'])
    if index%20 == 0:
        SolutionDF.to_csv(SAVE_PATH,index=False)
SolutionDF.to_csv(SAVE_PATH,index=False)

77 / 1400 

 #include <bits/stdc++.h>
using namespace std;
typedef long long ll;

int main() {
    int n, m, p;
    cin >> n >> m >> p;

    vector<int> a(n);
    for (int i = 0; i < n; ++i) {
        cin >> a[i];
    }

    vector<int> b(m);
    for (int i = 0; i < m; ++i) {
        cin >> b[i];
    }

    int i = 0;
    while (i < n && a[i] % p == 0) {
        i++;
    }

    int j = 0;
    while (j < m && b[j] % p == 0) {
        j++;
    }

    cout << i + j << '\n';
    return 0;
}
{'writer': 'AI', 'problem_id': '1316C', 'submission_id': nan, 'website': 'https://codeforces.com', 'source': "#include <bits/stdc++.h>\nusing namespace std;\ntypedef long long ll;\n\nint main() {\n    int n, m, p;\n    cin >> n >> m >> p;\n\n    vector<int> a(n);\n    for (int i = 0; i < n; ++i) {\n        cin >> a[i];\n    }\n\n    vector<int> b(m);\n    for (int i = 0; i < m; ++i) {\n        cin >> b[i];\n    }\n\n    int i = 0;\n    while (i < n && a[i] % p == 0) {\n        i++;\n    }\n\n    int j =

## Deepseek-Chat

In [ ]:
# Generate Solution from Programming.in.th
SolutionDF = pd.DataFrame(columns=['writer', 'problem_id', 'submission_id', 'website', 'source'])
from IPython.display import clear_output
import time
SAVE_PATH = './Generated_Solution/Pxsit_Suggested/DeepSeek-Chat_solV4-1.csv' #Change save PATH here
start_idx = 0
end_idx = 1500

for index, row in Combined_CF[:end_idx].iterrows():
    if index < start_idx:
        continue
    error_cnt = 0
    while True:
        if error_cnt >= 2:
            ans = None
            break
        ans, res = DeepseekGenerateSol(row['problem_statement'],promptV4) # Change Prompt Here
        if ans is not None:
            break
        else:
            print(res)
        if not ans == None:  
                break
        else:  
            error_message = str(res)
            if ('The document has no pages.' in error_message):
                print(f"Statement : {row['PdfURL']} not found")
                break
            print(f"ERROR : {error_message}")
            print(f"Failed at index {index}, retrying in 30 seconds...")
            SolutionDF.to_csv(SAVE_PATH,index=False)
            time.sleep(10)
            error_cnt+=1
    if ans == None:
        continue
    clear_output()
    print(f"{index} / {end_idx} ")
    print("\n", ans)
    SolutionInsert(ans, 'https://codeforces.com', row['problem_id'])
    if index%20 == 0:
        SolutionDF.to_csv(SAVE_PATH,index=False)
SolutionDF.to_csv(SAVE_PATH,index=False)

636 / 1500 

 #include<bits/stdc++.h>
using namespace std;
typedef long long ll;

int main() {
    int n, k, l;
    cin >> n >> k >> l;
    
    vector<bool> target(n + 2, false);
    for (int i = 0; i < k; ++i) {
        int x;
        cin >> x;
        target[x] = true;
    }
    
    vector<int> a(l);
    for (int i = 0; i < l; ++i) {
        cin >> a[i];
    }
    
    vector<bool> current(n + 2, false);
    queue<pair<vector<bool>, int>> q;
    q.emplace(current, 0);
    
    map<vector<bool>, int> visited;
    visited[current] = 0;
    
    int ans = -1;
    while (!q.empty()) {
        auto [state, steps] = q.front();
        q.pop();
        
        if (state == target) {
            ans = steps;
            break;
        }
        
        for (int i = 0; i < l; ++i) {
            int len = a[i];
            for (int j = 1; j <= n - len + 1; ++j) {
                vector<bool> next_state = state;
                for (int m = j; m < j + len; ++m) {
                    next_st